# Ejemplos de RDFlib

Ejemplos de como usar rdflib para representar conceptos

Creamos un grafo de conceptos vacío y un espacio de nombres que usaremos para los ejemplos

In [1]:
from rdflib.namespace import RDF, RDFS, Namespace, FOAF, OWL
from rdflib import Graph, BNode, Literal

g = Graph()

n = Namespace('http://ejemplo.org/')
g.bind('ejemplo', n)
g.bind('FOAF', FOAF)
g.bind('OWL', OWL)

Definimos un recurso que sea una persona

In [2]:
p1 = n.pedro
p1

rdflib.term.URIRef(u'http://ejemplo.org/pedro')

In [3]:
g.add((p1, RDF.type, FOAF.person))

Le añadimos un atributo edad con un valor que sera de la clase literal

In [4]:
v = Literal(22)
v

rdflib.term.Literal(u'22', datatype=rdflib.term.URIRef(u'http://www.w3.org/2001/XMLSchema#integer'))

In [5]:
g.add((p1, FOAF.age, v))

Podemos ver el resultado en RDF (XML)

In [6]:
print g.serialize(format='xml')

<?xml version="1.0" encoding="UTF-8"?>
<rdf:RDF
   xmlns:FOAF="http://xmlns.com/foaf/0.1/"
   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
>
  <rdf:Description rdf:about="http://ejemplo.org/pedro">
    <rdf:type rdf:resource="http://xmlns.com/foaf/0.1/person"/>
    <FOAF:age rdf:datatype="http://www.w3.org/2001/XMLSchema#integer">22</FOAF:age>
  </rdf:Description>
</rdf:RDF>



O en formato *turtle* que es un poco mas legible

In [7]:
print g.serialize(format='turtle')

@prefix FOAF: <http://xmlns.com/foaf/0.1/> .
@prefix OWL: <http://www.w3.org/2002/07/owl#> .
@prefix ejemplo: <http://ejemplo.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ejemplo:pedro a FOAF:person ;
    FOAF:age 22 .




Añadimos algunos hechos más al grafo

In [8]:
g.add((n.Barcelones, RDF.type, n.Europeo))
g.add((n.Europeo, RDF.type, OWL.Class))
g.add((n.Barcelones, RDFS.subClassOf, n.Europeo))

In [9]:
g.add((p1, RDF.type, n.Barcelones))
g.add((p1, FOAF.name, Literal('Pedro')))

Y este es el resultado

In [10]:
print g.serialize(format='turtle')

@prefix FOAF: <http://xmlns.com/foaf/0.1/> .
@prefix OWL: <http://www.w3.org/2002/07/owl#> .
@prefix ejemplo: <http://ejemplo.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ejemplo:pedro a ejemplo:Barcelones,
        FOAF:person ;
    FOAF:age 22 ;
    FOAF:name "Pedro" .

ejemplo:Barcelones a ejemplo:Europeo ;
    rdfs:subClassOf ejemplo:Europeo .

ejemplo:Europeo a OWL:Class .




Ahora podemos consultar cosas sobre cual es el tipo de Pedro

In [11]:
res = g.triples((n.pedro, RDF.type, None))
for s, p, o in res:
    print s, p, o

http://ejemplo.org/pedro http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://xmlns.com/foaf/0.1/person
http://ejemplo.org/pedro http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://ejemplo.org/Barcelones


Una clausura transitiva permite seguir la secuencia de definiciones de pedro

In [12]:
res = g.transitive_objects(n.pedro, RDF.type)
for r in res:
    print r

http://ejemplo.org/pedro
http://xmlns.com/foaf/0.1/person
http://ejemplo.org/Barcelones
http://ejemplo.org/Europeo
http://www.w3.org/2002/07/owl#Class


Tambén podemos usar el operador *in* para consultar si algo esta en el grafo

In [13]:
(n.pedro, None, None) in g # ¿Habla de Pedro el grafo?

True

In [14]:
(n.pedro, FOAF.age, None) in g # ¿Tiene una edad asignada?

True

In [15]:
(n.pedro, RDF.type, n.Barcelones) in g # ¿Es Pedro Barcelones?

True

También podemos utilizar SPARQL para hacer las consultas ...

In [16]:
res = g.query("""
            SELECT ?n ?e
            WHERE { 
                ?p foaf:age ?e.
                ?p foaf:name ?n.
            }
            """
              , initNs={'foaf': FOAF})
for nm, e in res:
    print nm, e

Pedro 22


Y modificaciones en el grafo

In [17]:
g.update("""
         INSERT DATA
         {
          ejemplo:maria a foaf:person; foaf:name 'Maria'; foaf:edad 27.
         }
            
""", initNs={'foaf': FOAF, 'ejemplo': n})

In [18]:
print g.serialize(format='turtle')

@prefix FOAF: <http://xmlns.com/foaf/0.1/> .
@prefix OWL: <http://www.w3.org/2002/07/owl#> .
@prefix ejemplo: <http://ejemplo.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ejemplo:maria a FOAF:person ;
    FOAF:edad 27 ;
    FOAF:name "Maria" .

ejemplo:pedro a ejemplo:Barcelones,
        FOAF:person ;
    FOAF:age 22 ;
    FOAF:name "Pedro" .

ejemplo:Barcelones a ejemplo:Europeo ;
    rdfs:subClassOf ejemplo:Europeo .

ejemplo:Europeo a OWL:Class .


